In [1]:
import pandas as pd
import numpy as np
import joblib
import random
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import make_scorer, mean_squared_error, r2_score
from itertools import combinations
from scipy import stats

In [2]:
data = pd.read_excel("/home/jovyan/CSCI 5700/Vert_data.xlsx")
data = data.iloc[:-5]

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 153 entries, 0 to 152
Data columns (total 27 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ID         153 non-null    object 
 1   Sex        153 non-null    object 
 2   Age_mean   153 non-null    object 
 3   C2         153 non-null    float64
 4   C3         153 non-null    float64
 5   C4         153 non-null    float64
 6   C5         153 non-null    float64
 7   C6         153 non-null    float64
 8   C7         153 non-null    float64
 9   T1         153 non-null    float64
 10  T2         153 non-null    float64
 11  T3         153 non-null    float64
 12  T4         153 non-null    float64
 13  T5         153 non-null    float64
 14  T6         153 non-null    float64
 15  T7         153 non-null    float64
 16  T8         153 non-null    float64
 17  T9         153 non-null    float64
 18  T10        153 non-null    float64
 19  T11        153 non-null    float64
 20  T12       

In [4]:
genders = ['M', 'F', 'UD']
results = {}
r2_results = {}
se_results = {}

data_combined = pd.concat([data[data['Sex'] == gender] for gender in genders])

y_combined = data_combined['Sum_Verts']
X_combined = data_combined.drop(columns=['Sum_Verts', 'ID', 'Age_mean', 'Sex'])

for gender in genders:
    y = data[data['Sex'] == gender]['Sum_Verts']
    X = data[data['Sex'] == gender].drop(columns=['Sum_Verts', 'ID', 'Age_mean', 'Sex'])
    
    rf = RandomForestRegressor(n_estimators=100, random_state=42)
    rf.fit(X, y)
    model_filename = f"random_forest_{gender}.joblib"
    joblib.dump(rf, model_filename)

spine_features = X_combined.columns.tolist()

for num_features in range(1, len(spine_features) + 1):
    r2_values = []
    se_values = []

    for _ in range(5):  # Run 10 times
        sampled_features = random.sample(spine_features, num_features)
        rf = RandomForestRegressor(n_estimators=100, random_state=42)
        
        rf.fit(X_combined[sampled_features], y_combined)
        predictions = rf.predict(X_combined[sampled_features])
        
        r2_value = r2_score(y_combined, predictions)
        r2_values.append(r2_value)
        
        se = np.std(predictions) / np.sqrt(len(predictions))  # Standard Error formula
        se_values.append(se)

    # Store the minimum R² and SE for this number of features
    r2_results[num_features] = np.min(r2_values)
    se_results[num_features] = np.min(se_values)

joblib.dump(r2_results, 'r2_results.joblib')
joblib.dump(se_results, 'se_results.joblib')


['se_results.joblib']

In [5]:
numeric_columns = data.select_dtypes(include=[np.number]).columns
data_numeric = data[numeric_columns]

In [6]:
### save the mean value
mean_values_by_gender = data.groupby('Sex')[numeric_columns].mean()
overall_mean = data[numeric_columns].mean()
mean_values_dict = pd.DataFrame({
    'M': mean_values_by_gender.loc['M'],
    'F': mean_values_by_gender.loc['F'],
    'UD': overall_mean    ####### Here we use the mean of overall to fill the nan value for the UD gender.
})
mean_values_dict = mean_values_dict.T

In [7]:
joblib.dump(mean_values_dict, "mean_values.joblib")

['mean_values.joblib']

In [8]:
mean_values_by_gender = joblib.load("mean_values.joblib")
mean_values_by_gender

,C2,C3,C4,C5,C6,C7,T1,T2,T3,T4,...,T9,T10,T11,T12,L1,L2,L3,L4,L5,Sum_Verts
M,38.350133,14.443867,13.702400,13.338533,13.524800,15.188400,17.455333,18.846267,18.731067,19.045333,...,21.604000,22.171867,22.878533,24.259200,25.609600,26.035200,27.088800,27.692800,27.533067,489.106800
F,35.174769,12.663538,12.220769,11.939077,12.202000,14.031692,15.561385,17.004462,16.965385,17.307692,...,19.559385,20.143077,20.943692,22.727846,23.837077,24.794923,25.333231,25.639385,25.346462,447.286462
UD,36.916078,13.659216,13.057778,12.706928,12.935098,14.669739,16.573137,17.998889,17.957124,18.272157,...,20.720719,21.331503,22.046993,23.565033,24.824314,25.482614,26.328301,26.791176,26.505359,470.556078


#### After the model, R², SE and mean value are saved, they can be directly called in subsequent use

In [51]:
mean_values = X_combined.mean()
std_values = X_combined.std()
max_values = round((mean_values + 2 * std_values),2)
min_values = round((mean_values - 2 * std_values),2)

def predict_sum_verts(user_input, user_gender):
    if user_gender not in ['M', 'F', 'UD']:
        raise ValueError("Invalid gender! Please input 'M', 'F', or 'UD'.")
#####################################################################################################################
    ##### If you need to contorl the input to have different decimal places, please use this part
    
    # for key in user_input:
    #     if not pd.isnull(user_input[key]): 
    #         user_input[key] = round(float(user_input[key]), 2)     #### Convert input into two decimal palces.
#####################################################################################################################
    
    non_null_features = [key for key, value in user_input.items() if not pd.isnull(value)]

    num_non_null_features = len(non_null_features)
    if num_non_null_features < 14:
        print("Warning: Insufficient data. Please provide at least 14 non-null features for accurate prediction.")
    
    gender_means = mean_values_by_gender.loc[user_gender]
    
    spine_features = ['C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'T1', 'T2', 'T3', 'T4', 'T5', 'T6',
                      'T7', 'T8', 'T9', 'T10', 'T11', 'T12', 'L1', 'L2', 'L3', 'L4', 'L5']

    for feature in spine_features:
        if feature not in user_input or pd.isnull(user_input[feature]):
            user_input[feature] = gender_means[feature]
        else:
            if user_input[feature] < min_values[feature] or user_input[feature] > max_values[feature]:
                print(f"Warning: The value for '{feature}' is out of bounds! "
                      f"Expected range: [{min_values[feature]}, {max_values[feature]}], "
                      f"but got: {user_input[feature]}.")

    input_df = pd.DataFrame([user_input], columns=spine_features)

    model_filename = f"random_forest_{user_gender}.joblib"
    model = joblib.load(model_filename)

    prediction = model.predict(input_df)

    # R² and SE
    r_squared = r2_results.get(num_non_null_features, None)
    se = se_results.get(num_non_null_features, None)

    predictions = model.predict(X_combined[spine_features])  
    std_dev = np.std(predictions)
    critical_value = stats.t.ppf(0.975, df=len(predictions) - 1)
    margin_of_error = critical_value * (std_dev / np.sqrt(len(predictions)))

    lower_bound = prediction[0] - margin_of_error
    upper_bound = prediction[0] + margin_of_error

    return round(prediction[0], 1), round(r_squared,4), round(se,4), (round(lower_bound, 1), round(upper_bound, 1))

### Test part

In [55]:
# def mask_user_input(user_input, mask_fraction=0.3):
#     features_to_mask = random.sample(list(user_input.keys()), int(len(user_input) * mask_fraction))
#     for feature in features_to_mask:
#         user_input[feature] = np.nan
#     return user_input

# all_mse = []
# for index, test_user in data.iterrows():
#     test_gender = test_user['Sex']  
#     user_input_data = {
#         'C2': test_user['C2'],
#         'C3': test_user['C3'],
#         'C4': test_user['C4'],
#         'C5': test_user['C5'],
#         'C6': test_user['C6'],
#         'C7': test_user['C7'],
#         'T1': test_user['T1'],
#         'T2': test_user['T2'],
#         'T3': test_user['T3'],
#         'T4': test_user['T4'],
#         'T5': test_user['T5'],
#         'T6': test_user['T6'],
#         'T7': test_user['T7'],
#         'T8': test_user['T8'],
#         'T9': test_user['T9'],
#         'T10': test_user['T10'],
#         'T11': test_user['T11'],
#         'T12': test_user['T12'],
#         'L1': test_user['L1'],
#         'L2': test_user['L2'],
#         'L3': test_user['L3'],
#         'L4': test_user['L4'],
#         'L5': test_user['L5'],
#     }
#     actual_sum_verts = test_user['Sum_Verts']
#     masked_user_input = mask_user_input(user_input_data)
#     predicted_sum_verts,r_squared,standard_error,_ = predict_sum_verts(masked_user_input, test_gender)
#     mse = mean_squared_error([actual_sum_verts], [predicted_sum_verts])
#     all_mse.append(mse)
# average_mse = np.mean(all_mse)
# print(f"Average Mean Squared Error across all users: {average_mse}")
# print(f"R_squared across all users: {r_squared}")
# print(f"Standard error across all users: {standard_error}")

In [56]:
# test_user = data.iloc[0]  
# test_gender = test_user['Sex'] 

# user_input_data = {
#     'C2': test_user['C2'],
#     'C3': test_user['C3'],
#     'C4': test_user['C4'],
#     'C5': test_user['C5'],
#     'C6': test_user['C6'],
#     'C7': test_user['C7'],
#     'T1': test_user['T1'],
#     'T2': test_user['T2'],
#     'T3': test_user['T3'],
#     'T4': test_user['T4'],
#     'T5': test_user['T5'],
#     'T6': test_user['T6'],
#     'T7': test_user['T7'],
#     'T8': test_user['T8'],
#     'T9': test_user['T9'],
#     'T10': test_user['T10'],
#     'T11': test_user['T11'],
#     'T12': test_user['T12'],
#     'L1': test_user['L1'],
#     'L2': test_user['L2'],
#     'L3': test_user['L3'],
#     'L4': test_user['L4'],
#     'L5': test_user['L5'],
# }



# actual_sum_verts = test_user['Sum_Verts']
# predicted_sum_verts,_,_,_= predict_sum_verts(user_input_data, test_gender)
# mse = mean_squared_error([actual_sum_verts], [predicted_sum_verts])
# print(f"Actual Sum_Verts: {actual_sum_verts}")
# print(f"Predicted Sum_Verts: {predicted_sum_verts}")
# print(f"Mean Squared Error: {mse}")


In [59]:
user_input_example = {
    'C2': 41.5,
    'C3': 13.6,
    'T5': 22.3,
    'L1': 16.9,
    'L2': 16.9,
    'L3': 14.9,
    'L4': 24,
    'L5': 24,
}

user_gender_example = 'M'  

predicted_sum_verts, r_squared, standard_errors, confidence_interval = predict_sum_verts(user_input_example, user_gender_example)
print(f"Predicted Sum_Verts for the user: {predicted_sum_verts}")
print(f"R_squared for the user: {r_squared}")
print(f"Standard error for the user: {standard_errors}")
print(f"95% Confidence Interval: {confidence_interval}")

Predicted Sum_Verts for the user: 487.6
R_squared for the user: 0.9879
Standard error for the user: 2.1741
95% Confidence Interval: (484.6, 490.6)
